# The goal is to find an apartment that is below its predicted price. That is to say, an apartment that is quantifiably 'a good deal'.

In [1]:
import pandas as pd
import cufflinks as cf
import sklearn
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
import plotly.express as px

TEST_SIZE = 0
BUDGET = 1300
REMOVE_OUTLIERS = True
# 5% because 95% percent of data is within 2 stddevs in gaussian distribution
OUTLIER_PERCENT = 20

if REMOVE_OUTLIERS:
    assert OUTLIER_PERCENT > 0, "Select a percentage of outliers to remove!"

cf.go_offline()
df = pd.DataFrame(pd.read_excel('apartments.xlsx'))
display(df)
df.drop(columns=['furnished', 'move_in_date'], inplace=True)

,ad_id,unit_type,bedrooms,bathrooms,hydro,heat,water,internet,cable,num_utils,...,washer_drier,dishwasher,a_c,num_appliances,outdoor_space,smoking,mins_guelph,mins_waterloo,price,available
0,1494496767,basement,1.0,1,1,1,1,1,0,4,...,0,0,1,2,1,0,10,28,1200,1
1,1495630096,duplex,1.0,1,0,1,1,0,0,2,...,1,0,1,3,1,0,20,19,1300,1
2,1494306061,"shared, basement",1.0,1,0,0,0,0,0,0,...,1,0,1,3,1,1,5,37,900,1
3,1495004948,apartment,1.0,1,0,1,1,0,0,2,...,0,0,0,0,1,0,8,29,1550,1
4,1493993007,apartment,1.0,1,0,1,1,0,0,2,...,0,0,0,0,1,0,8,29,1675,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,10,apartment,2.0,1,0,1,1,0,0,2,...,1,0,0,2,1,0,8,29,1405,1
66,1494328132,basement,1.0,1,1,1,1,0,0,3,...,1,1,1,4,0,0,7,31,1300,1
67,1495315557,apartment,2.0,1,0,1,1,0,0,2,...,1,0,0,2,1,1,4,32,1505,1
68,1490940151,apartment,2.0,1,0,1,1,0,0,2,...,0,0,0,1,1,0,7,29,1600,1


### Note: The original data source did not have num_utils, num_appliances and num_extras. They are composite attributes that I made as such:

#### num_utils are hydro, heat, water, internet, tv
#### num_appliances are fridge, washer&dryer, a/c, dishwasher
#### num_extras are pool, gym, elevator

### I did this because these features seem to be more useful as a single attribute than as several boolean values

### 2nd Note: the features 'mins_guelph' and 'mins_waterloo', were added more for personal reasons than for analyitical reasons but they may prove to be useful later.

In [2]:
def summary(df):
    ret = {}
    ret['null'] = df.isnull().sum()
    ret['null_avg'] = df.isnull().mean()
    ret['dtypes'] = df.dtypes
    ret['count'] = df.count()
    ret['std_dev'] = df.std()
    ret['mean'] = df.mean()
    ret['median'] = df.median()
    ret['min'] = df.min()
    ret['max'] = df.max()
    return pd.DataFrame(ret)

display(summary(df))
print(df.dtypes.value_counts())


,null,null_avg,dtypes,count,std_dev,mean,median,min,max
Fridge,0,0.000000,int64,70,3.524537e-01,8.571429e-01,1.000000e+00,0,1
a_c,0,0.000000,int64,70,4.866755e-01,3.714286e-01,0.000000e+00,0,1
ad_id,0,0.000000,int64,70,1.792767e+08,1.461680e+09,1.492312e+09,10,1496005699
agreement,0,0.000000,int64,70,2.571705e+00,1.137143e+01,1.200000e+01,1,12
available,0,0.000000,int64,70,1.195229e-01,9.857143e-01,1.000000e+00,0,1
bathrooms,0,0.000000,int64,70,3.468283e-01,1.100000e+00,1.000000e+00,1,3
bedrooms,0,0.000000,float64,70,8.746487e-01,1.714286e+00,2.000000e+00,1,6
cable,0,0.000000,int64,70,1.195229e-01,1.428571e-02,0.000000e+00,0,1
days_until_move_in,0,0.000000,int64,70,2.028076e+01,3.771429e+00,0.000000e+00,-100,49
dishwasher,0,0.000000,int64,70,4.615663e-01,3.000000e-01,0.000000e+00,0,1


int64      27
float64     2
object      1
dtype: int64


## Change non-categorical numeric fields to float and drop fields that were grouped into  a composite field

In [3]:
df = df.drop(columns = ['hydro', 'heat', 'water',
                   'internet', 'cable', 'gym', 'pool', 'elevator',
                   'Fridge', 'washer_drier', 'dishwasher', 'a_c'])
    
change_int_to_float = ['bathrooms', 'bedrooms','num_appliances', 'num_extras','num_utils', 'mins_guelph', 'mins_waterloo',
                      'price', 'size']

for feat in change_int_to_float:
    df[feat] = df[feat].astype('float64')
    

## Originally about half of the ads did not specify 'size'. Most of them could be found on the realtor websites, but there are still some missing values (because I could not find on the websites)
### To fix this I tried a mean imputation

In [4]:
def impute_size(row):
    if str(row['size']) == 'nan':
        row['size'] = df['size'].mean()
        
    return row

df = df.apply(impute_size, axis = 1)
display(summary(df))

,null,null_avg,dtypes,count,std_dev,mean,median,min,max
ad_id,0,0.0,int64,70,1.792767e+08,1.461680e+09,1.492312e+09,10,1496005699
agreement,0,0.0,int64,70,2.571705e+00,1.137143e+01,1.200000e+01,1,12
available,0,0.0,int64,70,1.195229e-01,9.857143e-01,1.000000e+00,0,1
bathrooms,0,0.0,float64,70,3.468283e-01,1.100000e+00,1.000000e+00,1,3
bedrooms,0,0.0,float64,70,8.746487e-01,1.714286e+00,2.000000e+00,1,6
days_until_move_in,0,0.0,int64,70,2.028076e+01,3.771429e+00,0.000000e+00,-100,49
mins_guelph,0,0.0,float64,70,1.118477e+01,1.612857e+01,1.050000e+01,3,54
mins_waterloo,0,0.0,float64,70,1.025719e+01,2.548571e+01,2.900000e+01,4,51
num_appliances,0,0.0,float64,70,1.190455e+00,2.214286e+00,2.000000e+00,0,4
num_extras,0,0.0,float64,70,9.223474e-01,7.000000e-01,0.000000e+00,0,3


## Normalize data for outlier detections with DBSCAN __TODO
Even though regression is invariant to scaling, DBSCAN is not and removing outliers may help our overall model

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
categorical = ['ad_id', 'unit_type']
cat = df.loc[:, categorical]
num = df.drop(columns=categorical)
# norm = pd.concat([pd.DataFrame(MinMaxScaler(feature_range=(1e-3, 1+(1e-3))).fit_transform(num), columns = num.columns), cat], axis = 1)

norm_df = pd.DataFrame(RobustScaler().fit_transform(num), columns = num.columns)


# remove outliers, while maintaining clusters at a minimum
num_outliers = OUTLIER_PERCENT*df.shape[0]//100
print('Searching for {} Outliers'.format(num_outliers))
    
outliers = None

smallest_clusters = float('inf')

lo_ep, hi_ep = 1, 20
# to get from lower to higher in intervals of 0.25
interval_ep = hi_ep//lo_ep*4 + 1

lo_ms, hi_ms = 3, 6


for ep in np.linspace(lo_ep, hi_ep, interval_ep):
    for ms in np.linspace(lo_ms, hi_ms, 4):
        db = DBSCAN(metric='manhattan', eps=ep, min_samples=ms).fit(norm_df)
        # update outliers
        if len(db.labels_) - len(db.labels_[db.labels_ >= 0]) == num_outliers and len(set(db.labels_)) < smallest_clusters:
            outliers = [i for i, e in enumerate(db.labels_) if e == -1]
            smallest_clusters = len(set(db.labels_))

assert outliers , 'Not enough outliers found using epsilon and min_samples parameters!'
print(outliers)

if REMOVE_OUTLIERS:
    try:
        df = df.drop(outliers)
        print('Removing {} Outliers'.format(num_outliers))
        outliers = None
    except:
        TypeError
        print('Outliers have already been removed')
    
display(summary(df))

Searching for 32 Outliers
[0, 1, 2, 8, 10, 12, 13, 14, 15, 18, 22, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40, 54, 55, 56, 57, 61, 66, 69]
Removing 32 Outliers


,null,null_avg,dtypes,count,std_dev,mean,median,min,max
ad_id,0,0.0,int64,38,2.418845e+08,1.437857e+09,1.486015e+09,10,1495315557
agreement,0,0.0,int64,38,0.000000e+00,1.200000e+01,1.200000e+01,12,12
available,0,0.0,int64,38,0.000000e+00,1.000000e+00,1.000000e+00,1,1
bathrooms,0,0.0,float64,38,2.262943e-01,1.052632e+00,1.000000e+00,1,2
bedrooms,0,0.0,float64,38,6.851400e-01,1.736842e+00,2.000000e+00,1,3
days_until_move_in,0,0.0,int64,38,4.063857e+00,-1.578947e-01,0.000000e+00,-12,18
mins_guelph,0,0.0,float64,38,9.786192e+00,1.547368e+01,8.000000e+00,3,31
mins_waterloo,0,0.0,float64,38,7.940614e+00,2.602632e+01,2.850000e+01,7,41
num_appliances,0,0.0,float64,38,1.119147e+00,1.868421e+00,2.000000e+00,0,4
num_extras,0,0.0,float64,38,9.440237e-01,1.026316e+00,1.000000e+00,0,3


## Info on the potential driving distances from each apartment

In [6]:
df[['mins_guelph', 'mins_waterloo']].iplot(kind='histogram', 
                                           xTitle='Distances in minutes to Univ. Guelph (Orange) /Waterloo (Blue)',
                                           title='Frequencies of Distances to Respective Universities')

In [7]:
MINS_WATERLOO_GUELPH = 33

travel_time = []
is_walking_distance = []
# if guelph is in walking distance, don't count it in daily travels
for i, row in df.iterrows():
    if row['mins_guelph'] > 5:
        time = row['mins_guelph']*2+MINS_WATERLOO_GUELPH*2  
        is_walking_distance.append(False)
    else:
        time = 2*row['mins_waterloo']
        is_walking_distance.append(True)
        
    travel_time.append(time)
    

travel_df = pd.DataFrame({'ad_id': df['ad_id'], 'daily_travel_dist' : travel_time, 'walkable' : is_walking_distance})
fig = px.histogram(travel_df,
                   x='daily_travel_dist',
                   color='walkable',
                   opacity=0.6,
                   hover_data=travel_df.columns,
                   marginal='box')
fig.show()

## More scatter plots between selected classes to observe relationships

In [8]:
drop_columns = ['num_extras', 'bathrooms','parking_spots', 'agreement', 'days_until_move_in',
       'pets', 'outdoor_space', 'smoking', 'mins_guelph', 'mins_waterloo', 'num_utils']

scatter_df = df.drop(columns=drop_columns)



for i, ca in enumerate(scatter_df.drop(columns=['ad_id', 'unit_type']).columns):
    for j, cb in enumerate(scatter_df.drop(columns=['ad_id', 'unit_type']).columns[i:]):
            if ca != cb:
                fig = px.scatter(scatter_df.loc[:, list(set(['ad_id', 'unit_type', ca, cb, 'price']))], 
                                 x=ca,
                                 y=cb,
                                 color='unit_type',
                                 title='{} vs {}'.format(ca, cb),
                                 size='price',
                                 hover_data=['ad_id'])

                fig.show()



## Plotting price and size, we can see there is a clear relationship between both attributes

In [9]:
df['price'].iplot(kind='histogram', xTitle='Price of Apartment', title='Frequencies of Apartment Prices')
df[['size']].iplot(kind='histogram', xTitle='Size of Apartment (SqFt)', title='Frequencies of Apartment Sizes in Sqft')

## Distribution of Amenities

In [10]:
df[['num_utils', 'num_extras', 'num_appliances']].iplot(kind='histogram', 
                                                    title='Number of Amenities by Groups')

In [11]:
df[['bedrooms']].iplot(kind='histogram')

In [12]:
import matplotlib.pyplot as plt 

layout1 = cf.Layout(
    height=900,
    width=1000
)
df.corr().iplot(kind='heatmap',
                colorscale="PuRd",
                title="Feature Correlation Matrix",
               layout=layout1)

## Simple Regression, Baseline Model

In [13]:
# scatter plot

train = df
fig = train.iplot(asFigure=True, 
                  x='size', 
                  y='price',
                  mode='markers',
                  title='Simple Scatter Plot to Visualize Usefulness of Linear Regression')
fig.show()

In [14]:
regressor = LinearRegression()
if TEST_SIZE:
    X_train, X_test, y_train, y_test = train_test_split(
        train.loc[:, ['size']],
        train.loc[:, ['price']],
        test_size=0.15,
        random_state=123
    )
else:
    
    X_train, y_train, = train.loc[:, ['size']], train.loc[:, ['price']]
    X_test, y_test = X_train, y_train
    
regressor.fit(X_train, y_train)

# #To retrieve the intercept:
print(regressor.intercept_)
# #For retrieving the slope:
print(regressor.coef_)

[576.49499242]
[[1.20945445]]


In [15]:
# X = np.array(np.linspace(min(train['size']), max(train['size']), 50))
y_pred = list( regressor.predict( X_test ).reshape(-1) )

test = pd.DataFrame({'x' : X_test['size'],
                          'y_test' : y_test['price'],
                          'y_pred' : y_pred
                    })

fig = go.Figure()


MSE = mean_squared_error(train['price'], test['y_pred'])
RMS = MSE**(1/2)
print('MSE', MSE, '\nRMS', RMS)

fig.update_layout(title='Regression Line Plotted vs Training Points | MSE={} |RMS={}'.format(MSE,RMS),
                  width=1000, height=600)
# draw training points
fig.add_trace(go.Scatter(x=train['size'], 
                         y=train['price'],
                         mode='markers',
                         name='Points',
                        ))


# draw regression line
fig.add_trace(go.Scatter(x=test['x'], y=test['y_pred'],mode='lines', name='Regression Line'))


fig.show()

MSE 23689.24282253088 
RMS 153.91310152982714


## Add More Features, Multiple Linear Regression

In [16]:
mult_regressor = LinearRegression()
features = ['bedrooms', 'num_appliances', 'num_utils', 'num_extras', 'size']
if TEST_SIZE:

    X_train, X_test, y_train, y_test = train_test_split(
        train.loc[:, features],
        train.loc[:, ['price']],
        test_size=0.15,
        random_state=123
    )
else:
    
    X_train, y_train = train.loc[:, features], train.loc[:, ['price']]
    X_test, y_test = X_train,  y_train

mult_regressor.fit(X_train, y_train)
# #To retrieve the intercept:
print(mult_regressor.intercept_)
# #For retrieving the slope:
print(mult_regressor.coef_)

[441.58986778]
[[ 10.13766396  19.35948955 -30.45447088  74.46367244   1.28725259]]


In [17]:
y_pred = list( mult_regressor.predict( X_test ).reshape(-1) )

mult_test = pd.DataFrame({'x' : X_test['size'],
                          'y_test' : y_test['price'],
                          'y_pred' : y_pred
                    })

fig = go.Figure()


MSE = mean_squared_error(y_test, y_pred)
RMS = MSE**(1/2)


fig.update_layout(title='Multiple Regression Predictions vs Ground Truths | MSE={} |RMS={}'.format(MSE,RMS),
                  width=1000, height=600)

fig.add_trace(go.Scatter(x=train.index, 
                         y=y_test['price'],
                         mode='markers+lines',
                         name='ground_truths'))

fig.add_trace(go.Scatter(x=train.index,
                         y=y_pred,
                         mode='markers+lines',
                         name='predictions'))



fig.show()



# While not an enormous difference, the mean squared error is lower when using the multiple regression as opposed to the simple linear regression
## Note that the line between the prediction points means nothing, it merely gives the graph more structure

# So which is the best priced one-bedroom apartment??

In [18]:
gain_df = pd.DataFrame({'benefit' : mult_test['y_test'] - mult_test['y_pred']})

final_df = pd.concat([gain_df, df], axis=1)
one_bedrooms_df = final_df[(final_df['bedrooms'] < 2.0) & (final_df['available'] == 1)]
within_budget = one_bedrooms_df[one_bedrooms_df['price'] < BUDGET]

print('Best Priced Bedroom Within Budget')
display(within_budget[within_budget['benefit'] == max(within_budget['benefit'])])

Best Priced Bedroom Within Budget


,benefit,ad_id,unit_type,bedrooms,bathrooms,num_utils,num_extras,parking_spots,agreement,days_until_move_in,pets,size,num_appliances,outdoor_space,smoking,mins_guelph,mins_waterloo,price,available
52,32.514837,1458068019,apartment,1.0,1.0,2.0,0.0,1,12,0,1,616.0,2.0,1,0,21.0,34.0,1255.0,1


In [20]:
# keeping track of RMS of Linear and Multiple Regression models according to the percentage of outliers we're finding
RMS_vs_OUTLIER_L = [(5, 185.09), (10, 178.85), (15, 165.05), (20, 154.72), (26, 153.39), (30, 155.33), 
                    (36, 150.248), (40, 150.09), (45, 153.91), (50, 153.53), (55, 152.10), (60, 115.84)]
RMS_vs_OUTLIER_M = [(5, 154.35), (10, 155.00), (15, 145.64), (20, 144.41), (26, 146.21), (30, 148.65), 
                    (36, 143.15), (40, 143.15),  (45, 140.77), (50, 148.55), (55, 134.84), (60, 107.22)]
xl, yl = zip(*RMS_vs_OUTLIER_L)
xm, ym = zip(*RMS_vs_OUTLIER_M)

fig = go.Figure()
fig.update_layout(height=600, width=1000)

fig.add_trace(go.Scatter(x=xl,
                         y=yl,
                         mode='markers+lines',
                         name='RMS of Linear vs Outlier Percentage'))

fig.add_trace(go.Scatter(x=xm,
                         y=ym,
                         mode='markers+lines',
                         name='RMS of M.R. vs Outlier Percentage'))

fig.show()

## TODO, compare the outliers that were removed from 26->36 and see if there were any noticeable patterns. Seeing as model improved a lot in theory on seen data, also take the outliers and use them as test set to see how model generalizes